In [1]:
import numpy as np
import pandas as pd
import bz2
from scipy.sparse import *
from scipy import *

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation

from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer
import string 

Using TensorFlow backend.


In [2]:
amazon_training = "./data/train.ft.txt.bz2"
amazon_testing = "./data/test.ft.txt.bz2"
# training_imdb = "./data/aclImdb/train"
# testing_imdb = "./data/aclImdb/test"

In [3]:
amazon_testing_file = bz2.BZ2File(amazon_testing,'r')
amazon_testing_lines = amazon_testing_file.readlines()

In [4]:
def parse_amazon_data(lines):
    '''
    :param lines: a list of string... each line being a review + label
    :return: a list of [review (string), label (int)]
    '''
    review_and_ratings_list=[]
    for i in range(len(lines)):
        line_text = str(lines[i])
        label, review_str = line_text.split(" ", 1)
        label = int(label[-1]) - 1         # label with 1 and 0
        review_and_ratings_list.append([review_str, label])
    return review_and_ratings_list

In [5]:
def remove_punc(s):
    table = str.maketrans({key: None for key in string.punctuation})
    return s.translate(table)

In [11]:
def generate_x_and_y(review_and_ratings_list):
    data = pd.DataFrame(review_and_ratings_list, columns=['review','rating'])
    data['word_count'] = data['review'].str.lower().str.split().apply(len)
    data['review'] = data['review'].apply(remove_punc)
    data = data[:][data['word_count']<=25]
    
    stop_word = text.ENGLISH_STOP_WORDS
    count_vector = CountVectorizer(stop_words = stop_word, min_df=.0001, lowercase=1)
    x = csr_matrix(count_vector.fit_transform(data['review'].values)).todense()
    y = csr_matrix(data['rating'].values).todense()
    print(x.shape)
    return (x, y)

In [12]:
amazon_testing_review_and_ratings_list = parse_amazon_data(amazon_testing_lines)

In [13]:
x, y = generate_x_and_y(amazon_testing_review_and_ratings_list)

(25768, 9678)


In [19]:
model1=  Sequential()
model1.add(Dense(1000,input_shape=(9678,),activation='relu'))
model1.add(Dense(25768,activation='sigmoid'))

model1.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
hist = model1.fit(x, y, epochs=6, batch_size=128, verbose=1)

ValueError: Input arrays should have the same number of samples as target arrays. Found 25768 input samples and 1 target samples.

In [ ]:
# model1.evaluate(X_test, y_test, batch_size=128)

In [ ]:
model1.evaluate(x, y, batch_size=128)